In [178]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src/')

import torch
import torch.optim as optim
from torchvision import datasets, transforms


from models.simple_LSTM import *
from models.train_model import train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [179]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [180]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: torch.mean(x, dim=2))
        ])

dataset = datasets.MNIST('../data/', download=True, train=True, transform=transform)
dataset.targets[dataset.targets % 2 == 0] = float(-1.0)
dataset.targets[dataset.targets % 2 == 1] = float(1.0)
train_set, test_set = torch.utils.data.random_split(dataset, [size, len(dataset)-size])

data_loader = torch.utils.data.DataLoader(dataset, batch_size = 1000)
size = int(len(dataset) * 0.8)


In [185]:
train_loader

In [181]:
image_size = 28
input_size = image_size * image_size
hidden_size = 10
num_layers = 2

model = SimpleLSTM(input_size, hidden_size, num_layers)
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')


The model has 32,731 trainable parameters


In [182]:
with torch.no_grad():
    print(f'Before training : label={test_loader[0][1]}, pred={model(test_loader[0][0])}')


Before training : label=1, pred=0.5006194114685059


In [183]:
train_loader

In [184]:
train(model, train_loader, optimizer, criterion, device, n_epochs=5)

Epoch 0:   0%|          | 0/48000 [00:00<?, ?batch/s]


AttributeError: 'int' object has no attribute 'to'

In [ ]:
with torch.no_grad():
    print(f'Before training : label={test_loader[0][1]}, pred={model(test_loader[0][0])}')


Before training : label=1, pred=0.4858831763267517
